In [1]:

"""
Connect senior:Mainland
"""
import pandas as pd
import json
import requests
def ssql(sql):
    admin_token = '2cebeb5c7edac5ef0d7c5a640e69fc7d43bf2cd6a24ce43dfee3dd33662c0bab'
    url = 'http://sensor.wb-intra.com/api/sql/query?token=%s&project=production' % admin_token
    data = {'q': sql, 'format': 'json'}
    req = requests.post(url,data)
    req_dec = req.content.decode()
    try:
        req_json = json.loads('[' + req_dec.replace('\n', ',')[:-1] + ']')
        df_d_id = pd.DataFrame(req_json)
        return df_d_id
    except:
        print(req_dec)

In [5]:
df = ssql("""

SELECT t3.date,
       t3.playerCount,
       t4.game_num,
       t4.duration
FROM
  (SELECT t2.date,
          t2.playerCount,
          t2.distinct_id
   FROM
     (SELECT date,distinct_id
      FROM events
      WHERE event = 'login'
        AND appId IN ('20014',
                      '30015')
        AND date BETWEEN '2020-10-20' AND current_date()- interval 1 DAY
      GROUP BY 1,
               2)t1
   LEFT JOIN
     (SELECT date,distinct_id,
                  playerCount
      FROM events
      WHERE event = 'gameStart'
        AND game_played = 0
        AND date BETWEEN '2020-10-20' AND current_date()- interval 1 DAY
      GROUP BY 1,
               2,
               3)t2 ON t1.distinct_id = t2.distinct_id
   AND t1.date = t2.date
   GROUP BY 1,
            2,
            3) t3
JOIN
  (SELECT date,distinct_id,
               count(distinct_id) AS game_num,
               sum(duration/60000)AS duration
   FROM events
   WHERE event = 'gameOver'
     AND gameTypeId = 1800
     AND date BETWEEN '2020-10-20' AND current_date()- interval 1 DAY
   GROUP BY 1,
            2)t4 ON t3.date = t4.date
AND t3.distinct_id = t4.distinct_id
WHERE t3.playerCount BETWEEN 4 AND 10
ORDER BY 1,
         2
""")

In [6]:
df

,date,duration,playercount,game_num
0,2020-10-20,37.183,4,7
1,2020-10-20,26.522,4,4
2,2020-10-20,113.201,4,17
3,2020-10-20,76.335,4,13
4,2020-10-20,3.819,4,1
...,...,...,...,...
4991733,2020-12-06,36.951,10,12
4991734,2020-12-06,10.840,10,3
4991735,2020-12-06,31.389,10,9
4991736,2020-12-06,32.488,10,9


In [7]:
df.groupby(['date','playercount'])['game_num','duration'].describe()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


game_num                                               \
                          count       mean        std  min  25%   50%    75%   
date       playercount                                                         
2020-10-20 4              310.0  10.322581  11.771523  1.0  2.0   6.0  13.00   
           5              314.0  10.764331  14.367669  1.0  2.0   6.0  12.75   
           6              403.0  10.622829  12.751809  1.0  2.0   6.0  14.00   
           7              343.0  10.574344  13.822587  1.0  2.0   6.0  13.50   
           8              527.0   8.929791  10.490119  1.0  2.0   5.0  12.00   
...                         ...        ...        ...  ...  ...   ...    ...   
2020-12-06 6            16551.0  17.457072  17.986499  1.0  5.0  12.0  24.00   
           7            10768.0  14.255015  15.962218  1.0  3.0   9.0  19.00   
           8            11595.0  14.038724  16.067188  1.0  3.0   9.0  19.00   
           9            11568.0  13.713779  16.255294  1.0  3.0   8.0  19.00   
           10           14055.0  13.597652  16.712909  1.0  3.0   7.0  18.00   

                              duration                                         \
                          max    count       mean        std    min       25%   
date       playercount                                                          
2020-10-20 4             85.0    310.0  57.550771  70.578062  0.535  10.61050   
           5            104.0    314.0  61.705799  85.910519  0.639  10.91900   
           6             77.0    403.0  63.057452  77.604243  0.781  11.69350   
           7            112.0    343.0  62.886633  82.654639  0.634  11.63600   
           8             77.0    527.0  54.614607  66.158740  0.781  11.62000   
...                       ...      ...        ...        ...    ...       ...   
2020-12-06 6            222.0  16551.0  64.629536  72.396847  0.029  16.44200   
           7            151.0  10768.0  51.703837  61.838745  0.357  11.56675   
           8            186.0  11595.0  55.653710  68.222941  0.651  12.71800   
           9            149.0  11568.0  57.394216  71.398641  0.355  12.34725   
           10           161.0  14055.0  58.401593  72.632281  0.632  12.90400   

                                                     
                            50%       75%       max  
date       playercount                               
2020-10-20 4            30.7795  80.43525   523.820  
           5            31.7745  69.68900   625.577  
           6            33.9010  81.80950   439.602  
           7            33.9010  82.17850   679.764  
           8            31.3030  71.37600   524.306  
...                         ...       ...       ...  
2020-12-06 6            41.2330  86.60500   883.224  
           7            31.4065  67.46250   733.842  
           8            33.0580  72.77350   856.964  
           9            32.6410  73.56975   855.242  
           10           33.0840  74.77850  1010.334  

[336 rows x 16 columns]

In [8]:
df.groupby(['date','playercount'])['game_num','duration'].describe().to_clipboard()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.
